#### What are you trying to do in this notebook?
For this challenge, we will be predicting a full year worth of sales for three items at two stores located in three different countries. This dataset is completely fictional, but contains many effects you see in real-world data, e.g., weekend and holiday effect, seasonality, etc. The dataset is small enough to allow us to try numerous different modeling approaches.

#### Why are you trying it?
There are two (fictitious) independent store chains selling Kaggle merchandise that want to become the official outlet for all things Kaggle. we want to figure out which of the store chains(KaggleMart or KaggleRama) would have the best sales going forward.

**Files**

- train.csv - the training set, which includes the sales data for each date-country-store-item combination.
- test.csv - the test set; your task is to predict the corresponding item sales for each date-country-store-item combination. Note the Public leaderboard is scored on the first quarter of the test year, and the Private on the remaining.
- sample_submission.csv - a sample submission file in the correct format.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%capture
!pip install pycaret[full]

In [ ]:
import pandas as pd
import numpy as np 
from pycaret.regression import *

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv',index_col='row_id')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv',index_col='row_id')

In [ ]:
def pre_process(df):
    
    df['date'] = pd.to_datetime(df['date'])
    df['week']= df['date'].dt.week
    df['year'] = 'Y'+df['date'].dt.year.astype(str)
    df['quarter'] = 'Q'+df['date'].dt.quarter.astype(str)
    df['day'] = df['date'].dt.day
    df['dayofyear'] = df['date'].dt.dayofyear
    df.loc[(df.date.dt.is_leap_year) & (df.dayofyear >= 60),'dayofyear'] -= 1
    df['weekend'] = df['date'].dt.weekday >=5
    df['weekday'] = 'WD' + df['date'].dt.weekday.astype(str)
    df.drop(columns=['date'],inplace=True)   

pre_process(train)
pre_process(test)

In [ ]:
train.info(), test.info()

In [ ]:
# Credit to https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36414
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

In [ ]:
reg = setup(data = train,
            target = 'num_sold',
            normalize=True,
            normalize_method='robust',
            transform_target = True,
            data_split_shuffle = False, #so that we do not use "future" observations to predict "past" observations
            create_clusters = False,
            use_gpu = True,
            silent = True,
            fold=10,
            n_jobs = -1)

In [ ]:
add_metric('SMAPE', 'SMAPE', SMAPE, greater_is_better = False)
top =compare_models(sort = 'SMAPE',n_select = 3, include = ['catboost','lightgbm','xgboost'])

In [ ]:
blend = blend_models(top)
predict_model(blend)

In [ ]:
final_blend = finalize_model(blend)
predict_model(final_blend)

In [ ]:
preds = predict_model(final_blend, data=test)
sub = pd.DataFrame(list(zip(test.index,preds.Label)),columns = ['row_id', 'num_sold'])
sub.to_csv('submission.csv', index = False)
print(sub.head(),sub.describe())

#### Did it work?
The notebook goes together with the EDA notebook, which visualizes the various seasonal effects and the differences in growth rate. Scikit-learn doesn't offer SMAPE as a loss function. As a workaround, I'm training for Huber loss with a transformed target, apply a correction factor, and we'll see how far we'll get.

The transformed target for the regression is the log of the sales numbers.

#### What did you not understand about this process?
Well, everything provides in the competition data page. I've no problem while working on it. If you guys don't understand the thing that I'll do in this notebook then please comment on this notebook.

What else do you think you can try as part of this approach?
Look at a notebook which presents feature engineering (based on the insights of this EDA) and a linear model which makes use of the features.

All credit goes to - https://www.kaggle.com/bernhardklinger/tps-jan-2022 

**I Hope you find this notebook useful , Good Luck!**